# 第5章: 係り受け解析

- CaboCha
https://taku910.github.io/cabocha/

```
brew install cabocha
```
で入った

In [1]:
!cabocha -v

cabocha of 0.69


In [2]:
!cat neko.txt | cabocha -f1 > neko.txt.cabocha

In [3]:
!head -n20 neko.txt.cabocha

* 0 -1D 0/0 0.000000
一	名詞,数,*,*,*,*,一,イチ,イチ
EOS
EOS
* 0 2D 0/0 -0.764522
　	記号,空白,*,*,*,*,　,　,　
* 1 2D 0/1 -0.764522
吾輩	名詞,代名詞,一般,*,*,*,吾輩,ワガハイ,ワガハイ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
* 2 -1D 0/2 0.000000
猫	名詞,一般,*,*,*,*,猫,ネコ,ネコ
で	助動詞,*,*,*,特殊・ダ,連用形,だ,デ,デ
ある	助動詞,*,*,*,五段・ラ行アル,基本形,ある,アル,アル
。	記号,句点,*,*,*,*,。,。,。
EOS
* 0 2D 0/1 -1.911675
名前	名詞,一般,*,*,*,*,名前,ナマエ,ナマエ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
* 1 2D 0/0 -1.911675
まだ	副詞,助詞類接続,*,*,*,*,まだ,マダ,マダ


## 40. 係り受け解析結果の読み込み（形態素）

In [4]:
class Morph:
    def __init__(self,mor):
        self.parse(mor)

    def __str__(self):
        return self.mor
        
    def parse(self,mor):
        self.mor = mor
        splited = [mor.split('\t')[0]] + mor.split('\t')[1].split(',')
        self.surface=splited[0]
        self.base=splited[7]
        self.pos=splited[1]
        self.pos1=splited[2]     


with open('neko.txt.cabocha') as f:
    all_text = []
    sentence_list = []
    for line in f:
        if line=='EOS\n':
            if len(sentence_list)>0:
                all_text.append(sentence_list[:])
            sentence_list = []
        elif line[0]=='*':
            pass
        else:
            sentence_list.append(Morph(line))


In [5]:
for i in range(3):
    for mor in all_text[i]:
        print(mor,end='')
    print()

一	名詞,数,*,*,*,*,一,イチ,イチ

　	記号,空白,*,*,*,*,　,　,　
吾輩	名詞,代名詞,一般,*,*,*,吾輩,ワガハイ,ワガハイ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
猫	名詞,一般,*,*,*,*,猫,ネコ,ネコ
で	助動詞,*,*,*,特殊・ダ,連用形,だ,デ,デ
ある	助動詞,*,*,*,五段・ラ行アル,基本形,ある,アル,アル
。	記号,句点,*,*,*,*,。,。,。

名前	名詞,一般,*,*,*,*,名前,ナマエ,ナマエ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
まだ	副詞,助詞類接続,*,*,*,*,まだ,マダ,マダ
無い	形容詞,自立,*,*,形容詞・アウオ段,基本形,無い,ナイ,ナイ
。	記号,句点,*,*,*,*,。,。,。



## 41. 係り受け解析結果の読み込み（文節・係り受け）

In [6]:
class Chunk:
    def __init__(self,ctxt):
        self.morphs = []
        self.srcs = []
        self.dst = -1
        self.parse(ctxt)
    
    def __str__(self):
        s = ""
        for m in self.morphs:
            s += m.surface
        return s
    
    def text(self):
        s = self.ctxt
        for m in self.morphs:
            s += m.__str__()
        return s        
       
    def parse(self,ctxt):
        self.ctxt = ctxt
        self.idx = int(ctxt.split()[1])
        self.dst = int(ctxt.split()[2][:-1])


In [7]:
all_text = []
sentence_list = []
srcs = {}
with open('neko.txt.cabocha') as f:
    for line in f:
        if line[0]=='*':
            c = Chunk(line)
            sentence_list.append(c)
            if c.dst in srcs.keys():
                srcs[c.dst].append(c.idx)
            else:
                srcs[c.dst] = [c.idx]
            if c.idx in srcs:
                c.srcs = srcs[c.idx][:]
        elif line=='EOS\n':
            srcs = {}
            if len(sentence_list)>0:
                all_text.append(sentence_list[:])
            sentence_list = []
        else:
            c.morphs.append(Morph(line))


In [8]:
for chunk in all_text[7]:
    print("{} {} -> {}".format(chunk.idx,chunk,chunk.dst))

0 この -> 1
1 書生というのは -> 7
2 時々 -> 4
3 我々を -> 4
4 捕えて -> 5
5 煮て -> 6
6 食うという -> 7
7 話である。 -> -1


## 42. 係り元と係り先の文節の表示

In [9]:
import re
for sentence in all_text[0:4]:
    for chunk in sentence:
        if chunk.dst == -1: continue
        print(re.sub('[、。　]','',"{}\t{}".format(chunk,sentence[chunk.dst])))

	猫である
吾輩は	猫である
名前は	無い
まだ	無い
どこで	生れたか
生れたか	つかぬ
とんと	つかぬ
見当が	つかぬ


## 43. 名詞を含む文節が動詞を含む文節に係るものを抽出

In [10]:
for sentence in all_text[0:10]:
    for chunk in sentence:
        if chunk.dst == -1: continue
        cfrom = chunk
        cto   = sentence[chunk.dst]
        ffrom = False
        fto   = False
        for mor in cfrom.morphs:
            if mor.pos=='名詞': ffrom = True
        for mor in cto.morphs:
            if mor.pos=='動詞': fto = True
        if ffrom and fto:
            print(re.sub('[、。　]','',"{}\t{}".format(cfrom,cto)))

どこで	生れたか
見当が	つかぬ
所で	泣いて
ニャーニャー	泣いて
いた事だけは	記憶している
吾輩は	見た
ここで	始めて
ものを	見た
あとで	聞くと
我々を	捕えて
掌に	載せられて
スーと	持ち上げられた
時	フワフワした
感じが	あったばかりである
